# Modelo de Gas en un Turborreactor

Resuelva un turborreactor de $0.5m^2$ de área de entrada, con difusor y tobera isoentrópicas, dos compresores de relación de compresión $5:1$ y rendimiento adiabático del $75\%$ acoplados a una única turbina de rendimiento $80\%$ y cuyo eje produce pérdidas por valor de un $8\%$ de la energía generada. La combustión con se produce a pérdida de carga despreciable y un rendimiento de la combustión del $95\%$ además de un ratio de equivalencia del $50\%$ con un combustible de $43\frac{MJ}{kg}$ y dosado estequiométrica $f=0.0664$.

Asuma velocidad de entrada de la corriente laminar de 200 $\frac{m}{s}$ y altura de vuelo de $5000m$.

Con todo esto obtenga las actuaciones y compare los resultados entre la aproximación de gas perfecto y la de gas semiperfecto. Establezca, en el modelo de gas semiperfecto, la evolución de calor específico y coeficiente de dilatación adiabático variables en cada estación.

In [19]:
# Auxiliar
etapa = "AUX"
print("{:5s}:  pt={:8.0f}Pa;  Tt={:4.0f}K;  G={:5.1f}kg/s;  Ht={:7.3f}MW".format(etapa, 1, 2, 3, 4*1e-6))

AUX  :  pt=       1Pa;  Tt=   2K;  G=  3.0kg/s;  Ht=  0.000MW


In [20]:
# Importaciones de código
from pyturb.gas_models import isa, PerfectIdealGas, SemiperfectIdealGas, IsentropicFlow
import numpy as np
import iteradores_gas_semiperfecto as iteradores

aire = SemiperfectIdealGas('Air')
aire_isent = IsentropicFlow(aire)

In [21]:
## Datos turborreactor (entrada)
Ae = 0.5 #m^2
h0 = 5000#m
v0 = 200 #m/s
# Difusor isoentrópico

# Tobero: desconocida el área, isoentrópica

# Datos turbomáquina
rend_c1 = rend_c2 = 0.75
pi_c1 = pi_c2 = 5
rend_cc = 0.95
pi_cc = 1
rend_t = 0.8
# Pérdidas en eje turbina del 8%

In [22]:
# Datos combustible
L = 43e6 # J/kg
fs = 0.0664
ratio_equiv = 0.5

In [23]:
## Entrada y difusor
# Entrada etapa 1:
T0 = isa.temperature_isa(h0)
p0 = isa.pressure_isa(h0)
rho0 = p0/T0/aire.Rg

G1=G0 = Ae*v0*rho0

M1=M0 = aire_isent.mach_number(v0, T0)
T1t=T0t = aire_isent.stag_temp_from_mach(M0, T0)
p1t=p0t = aire_isent.stag_pressure_from_mach(M0, p0, T0)

H1t = G1*aire.cp(T1t)*T1t

print("p0={:5.0f}Pa;  T0={:4.1f}K; M0={:4.3f}".format(p0, T0, M0))
etapa = "Dif_1"
print("{:5s}:  pt={:8.0f}Pa;  Tt={:4.0f}K;  G={:5.1f}kg/s;  Ht={:7.3f}MW".format(etapa, p1t, T1t, G1, H1t*1e-6))

# Salida del difusor 2:
T2t = T1t
p2t = p1t
H2t = H1t
G2 = G1

etapa = "Dif_2"
print("{:5s}:  pt={:8.0f}Pa;  Tt={:4.0f}K;  G={:5.1f}kg/s;  Ht={:7.3f}MW".format(etapa, p2t, T2t, G2, H2t*1e-6))

p0=54020Pa;  T0=255.6K; M0=0.624
Dif_1:  pt=   70231Pa;  Tt= 276K;  G= 73.6kg/s;  Ht= 20.364MW
Dif_2:  pt=   70231Pa;  Tt= 276K;  G= 73.6kg/s;  Ht= 20.364MW


$$ \dot{W}_c + \dot{Q_c} = \Delta\dot{H}_c = G c_p \left(T_{25t}-T_{2t}\right)$$
$$ \eta_c = \frac{\pi_c^{\frac{\gamma-1}{\gamma}}-1}{\frac{T_{25t}}{T_{2t}}-1}$$

$$ \pi_c^{\frac{\gamma-1}{\gamma}}-1 = \eta_c \left(\frac{T_{25t}}{T_{2t}}-1\right)$$

In [24]:
# Compresor baja
Tcomp1 = T2t
p25t = p2t * pi_c1

# Modelo perfecto e ideal
T25t_T2t_ideal = (pi_c1**((aire.gamma(Tcomp1)-1)/aire.gamma(Tcomp1)) - 1)/rend_c1 + 1
T25t_ideal = T25t_T2t_ideal * T2t

# Temperatura final:
T25t = iteradores.itera_compresor(rend_c1, pi_c1, T2t, aire)

G25 = G2
H25t = G25 * aire.cp(T25t) * T25t

etapa = "COMP1"
print("{:5s}:  pt={:8.0f}Pa;  Tt={:4.0f}K;  G={:5.1f}kg/s;  Ht={:7.3f}MW".format(etapa, p25t, T25t, G25, H25t*1e-6))

Wc1 = H25t-H2t

1 --  Tst= 490.3K;  T_c= 275.6K;  deltaT=0.745299;  Converge: False
2 --  Tst= 488.3K;  T_c= 383.0K;  deltaT=0.006999;  Converge: False
3 --  Tst= 488.4K;  T_c= 382.0K;  deltaT=0.000102;  Converge: False
4 --  Tst= 488.4K;  T_c= 382.0K;  deltaT=0.000001;  Converge: True
COMP1:  pt=  351154Pa;  Tt= 488K;  G= 73.6kg/s;  Ht= 36.931MW


In [25]:
# Compresor alta
Tcomp2 = T25t
p3t = p25t * pi_c2

# Temperatura final:
T3t = iteradores.itera_compresor(rend_c2, pi_c2, T25t, aire)

G3 = G25
H3t = G3 * aire.cp(T3t) * T3t

etapa = "COMP2"
print("{:5s}:  pt={:8.0f}Pa;  Tt={:4.0f}K;  G={:5.1f}kg/s;  Ht={:7.3f}MW".format(etapa, p3t, T3t, G3, H3t*1e-6))

Wc2 = H3t-H25t

1 --  Tst= 858.1K;  T_c= 488.4K;  deltaT=1.283176;  Converge: False
2 --  Tst= 840.6K;  T_c= 673.2K;  deltaT=0.060751;  Converge: False
3 --  Tst= 841.5K;  T_c= 664.5K;  deltaT=0.002988;  Converge: False
4 --  Tst= 841.4K;  T_c= 664.9K;  deltaT=0.000147;  Converge: False
5 --  Tst= 841.4K;  T_c= 664.9K;  deltaT=0.000007;  Converge: True
COMP2:  pt= 1755768Pa;  Tt= 841K;  G= 73.6kg/s;  Ht= 68.638MW


$$ \dot{W}_{cc} + \dot{Q_{cc}} = \Delta\dot{H}_{cc} = G c_p \left(T_{4t}-T_{3t}\right) = \eta_q c L$$

In [32]:
# Cámara de combustión
G4 = G3 # Hipótesis de combustión diluida
f = ratio_equiv*fs
c = G3*f
Q43 = rend_cc * c * L

T4t = iteradores.itera_combustor(Q43, G4, T3t, aire)
p4t = p3t * pi_cc
H4t = G4*aire.cp(T4t)*T4t

etapa = "COMB"
print("{:5s}:  pt={:8.0f}Pa;  Tt={:4.0f}K;  G={:5.1f}kg/s;  Ht={:7.3f}MW".format(etapa, p4t, T4t, G4, H4t*1e-6))

1 --  Tst=2065.3K;  T_c= 841.4K;  deltaT=4.247287;  Converge: False
2 --  Tst=1965.8K;  T_c=1453.3K;  deltaT=0.345077;  Converge: False
3 --  Tst=1970.9K;  T_c=1403.6K;  deltaT=0.017665;  Converge: False
4 --  Tst=1970.7K;  T_c=1406.2K;  deltaT=0.000934;  Converge: False
5 --  Tst=1970.7K;  T_c=1406.0K;  deltaT=0.000049;  Converge: False
6 --  Tst=1970.7K;  T_c=1406.0K;  deltaT=0.000003;  Converge: True
COMB :  pt= 1755768Pa;  Tt=1971K;  G= 73.6kg/s;  Ht=181.111MW


$$ W_{c1} + W_{c2} + W_{p} + W_{t1} = 0; \qquad W_{p} = -0.08W_{t1}$$

$$ W_{c1} + W_{c2} + W_{turb} = 0 \rightarrow W_{turb} = 1.08 \times W_{t1} = 1.08\times\left(-W_{c1}-W_{c2}\right)$$

$$ W_t = G c_p \left( T_{5t}-T_{4t}\right)$$

$$ \eta_t \times \left(\pi_t^{\frac{\gamma-1}{\gamma}} - 1\right) = \frac{T_{5t}}{T_{4t}} - 1$$

In [27]:
# Turbina
Wconsumidores = 1.08*(Wc1 + Wc2)
Wt = -Wconsumidores
G5 = G4
T5t = iteradores.itera_turbina(Wt, G5, T4t, aire)

H5t = G5*aire.cp(T5t)*T5t

Tturb = (T5t + T4t)/2
pi_t = ((T5t/T4t - 1)/ rend_t + 1)**(aire.gamma(Tturb)/(aire.gamma(Tturb)-1))
p5t = p4t*pi_t

etapa = "TURB"
print("{:5s}:  pt={:8.0f}Pa;  Tt={:4.0f}K;  G={:5.1f}kg/s;  Ht={:7.3f}MW".format(etapa, p5t, T5t, G5, H5t*1e-6))

1 --  Tst=1403.4K;  T_c=1970.7K;  deltaT=1.968748;  Converge: False
2 --  Tst=1393.9K;  T_c=1687.0K;  deltaT=0.032825;  Converge: False
3 --  Tst=1393.7K;  T_c=1682.3K;  deltaT=0.000637;  Converge: False
4 --  Tst=1393.7K;  T_c=1682.2K;  deltaT=0.000012;  Converge: False
5 --  Tst=1393.7K;  T_c=1682.2K;  deltaT=0.000000;  Converge: True
TURB :  pt=  250160Pa;  Tt=1394K;  G= 73.6kg/s;  Ht=123.073MW


Tobera isoentrópica: $\rightarrow \frac{T_9}{T_{9t}} = \left(\frac{p_9}{p_{9t}}\right)^{\frac{\gamma}{\gamma-1}}$

In [28]:
# Tobera
T9t = T8t = T7t = T5t
p9t = p8t = p7t = p5t
G9 = G8 = G7 = G5
H9t = H8t= H7t = H5t

p9 = p0
T9 = (p9/p9t)**((aire.gamma(T9t) - 1)/aire.gamma(T9t)) * T9t
rho9 = p9/T9/aire.Rg
                    
v9 = np.sqrt(2*aire.cp(T9t)*(T9t-T9))

M9 = aire_isent.mach_number(v9, T9)
A9 = G9/v9/rho9

etapa = "TOB"
print("{:5s}:  pt={:8.0f}Pa;  Tt={:4.0f}K;  G={:5.1f}kg/s;  Ht={:7.3f}MW".format(etapa, p9t, T9t, G9, H9t*1e-6))

print(M9, v9, T9, p9, A9)

TOB  :  pt=  250160Pa;  Tt=1394K;  G= 73.6kg/s;  Ht=123.073MW
1.663149605025283 1013.2427689542044 965.8138296627029 54019.912103762086 0.3728499676296921


In [29]:
# Actuaciones
E = G9*v9 - G0*v0 + A9 * (p9-p0)
ce = c/E
Isp = E/G9
print("E={0:5.1f}kN;  ce={1:7.3f}g/skN;  Isp={2:5.1f}m/s".format(E*1e-3, ce*1e6, Isp))

E= 59.9kN;  ce= 40.824g/skN;  Isp=813.2m/s


In [36]:
etapa = "Dif_1"
print("{:5s}:  pt={:8.0f}Pa;  Tt={:4.0f}K;  G={:5.1f}kg/s;  Ht={:7.3f}MW".format(etapa, p1t, T1t, G1, H1t*1e-6))

etapa = "Dif_2"
print("{:5s}:  pt={:8.0f}Pa;  Tt={:4.0f}K;  G={:5.1f}kg/s;  Ht={:7.3f}MW".format(etapa, p2t, T2t, G2, H2t*1e-6))

etapa = "COMP1"
print("{:5s}:  pt={:8.0f}Pa;  Tt={:4.0f}K;  G={:5.1f}kg/s;  Ht={:7.3f}MW".format(etapa, p25t, T25t, G3, H25t*1e-6))

etapa = "COMP2"
print("{:5s}:  pt={:8.0f}Pa;  Tt={:4.0f}K;  G={:5.1f}kg/s;  Ht={:7.3f}MW".format(etapa, p3t, T3t, G3, H3t*1e-6))

etapa = "COMB"
print("{:5s}:  pt={:8.0f}Pa;  Tt={:4.0f}K;  G={:5.1f}kg/s;  Ht={:7.3f}MW".format(etapa, p4t, T4t, G4, H4t*1e-6))

etapa = "TURB"
print("{:5s}:  pt={:8.0f}Pa;  Tt={:4.0f}K;  G={:5.1f}kg/s;  Ht={:7.3f}MW".format(etapa, p5t, T5t, G5, H5t*1e-6))

etapa = "TOB"
print("{:5s}:  pt={:8.0f}Pa;  Tt={:4.0f}K;  G={:5.1f}kg/s;  Ht={:7.3f}MW".format(etapa, p9t, T9t, G9, H9t*1e-6))

print(M9, v9, T9, p9, A9)

Dif_1:  pt=   70231Pa;  Tt= 276K;  G= 73.6kg/s;  Ht= 20.364MW
Dif_2:  pt=   70231Pa;  Tt= 276K;  G= 73.6kg/s;  Ht= 20.364MW
COMP1:  pt=  351154Pa;  Tt= 488K;  G= 73.6kg/s;  Ht= 36.931MW
COMP2:  pt= 1755768Pa;  Tt= 841K;  G= 73.6kg/s;  Ht= 68.638MW
COMB :  pt= 1755768Pa;  Tt=1971K;  G= 73.6kg/s;  Ht=181.111MW
TURB :  pt=  250160Pa;  Tt=1394K;  G= 73.6kg/s;  Ht=123.073MW
TOB  :  pt=  250160Pa;  Tt=1394K;  G= 73.6kg/s;  Ht=123.073MW
1.663149605025283 1013.2427689542044 965.8138296627029 54019.912103762086 0.3728499676296921


In [35]:
# Turborreactor completo:
def turborreactor(rho0, v0):
    G2 = G1=G0 = Ae*v0*rho0
    M1=M0 = aire_isent.mach_number(v0, T0)
    T2t = T1t=T0t = aire_isent.stag_temp_from_mach(M0, T0)
    p2t = p1t=p0t = aire_isent.stag_pressure_from_mach(M0, p0, T0)
    H1t = G1*aire.cp(T1t)*T1t

    # Compresor baja
    p25t = p2t * pi_c1
    T25t = iteradores.itera_compresor(rend_c1, pi_c1, T2t, aire)
    G25 = G2
    H25t = G25 * aire.cp(T25t) * T25t
    Wc1 = H25t-H2t

    # Compresor alta
    Tcomp2 = T25t
    p3t = p25t * pi_c2
    T3t = iteradores.itera_compresor(rend_c2, pi_c2, T25t, aire)
    G3 = G25
    H3t = G3 * aire.cp(T3t) * T3t
    Wc2 = H3t-H25t

    # Cámara de combustión
    G4 = G3 # Hipótesis de combustión diluida
    f = ratio_equiv*fs
    c = G3*f
    Q43 = rend_cc * c * L
    T4t = iteradores.itera_combustor(Q43, G4, T3t, aire)
    p4t = p3t * pi_cc
    H4t = G4*aire.cp(T4t)*T4t

    # Turbina
    Wconsumidores = 1.08*(Wc1 + Wc2)
    Wt = -Wconsumidores
    G5 = G4
    T5t = iteradores.itera_turbina(Wt, G5, T4t, aire)
    H5t = G5*aire.cp(T5t)*T5t
    Tturb = (T5t + T4t)/2
    pi_t = ((T5t/T4t - 1)/ rend_t + 1)**(aire.gamma(Tturb)/(aire.gamma(Tturb)-1))
    p5t = p4t*pi_t

    # Tobera
    T9t = T8t = T7t = T5t
    p9t = p8t = p7t = p5t
    G9 = G8 = G7 = G5
    H9t = H8t= H7t = H5t
    p9 = p0
    T9 = (p9/p9t)**((aire.gamma(T9t) - 1)/aire.gamma(T9t)) * T9t
    rho9 = p9/T9/aire.Rg
    v9 = np.sqrt(2*aire.cp(T9t)*(T9t-T9))
    M9 = aire_isent.mach_number(v9, T9)
    A9 = G9/v9/rho9
    
    return p3t, p4t, T5t, v9